In [3]:
import pandas as pd
from tqdm.notebook import tqdm
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point
import os

pd.set_option('display.max_columns', None)

tqdm.pandas()

In [2]:
df = pd.read_csv(r'D:\Download\wibrarchive (1).csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Download\\wibrarchive (1).csv'

In [14]:
coordinate_columns = df[['RoughX', 'RoughY']]
coordinate_columns.head()

,RoughX,RoughY
0,2548652.700,384878.988
1,2552783.522,394664.536
2,2558128.379,398704.395
3,2558178.583,379146.422
4,2540765.508,396454.197


In [15]:
transformer = Transformer.from_crs('epsg:32054', 'epsg:4326', always_xy=True)

def convert_coordinates(x, y):
    lon, lat = transformer.transform(x, y)
    return lon, lat

In [16]:
df['Longitude'], df['Latitude'] = zip(*df.progress_apply(lambda row: convert_coordinates(row['RoughX'], row['RoughY']), axis=1))

  0%|          | 0/847214 [00:00<?, ?it/s]

In [10]:
df.head()

,IncidentNum,ReportedDateTime,ReportedYear,ReportedMonth,Location,WeaponUsed,ALD,NSP,POLICE,TRACT,WARD,ZIP,RoughX,RoughY,Arson,AssaultOffense,Burglary,CriminalDamage,Homicide,LockedVehicle,Robbery,SexOffense,Theft,VehicleTheft,Longitude,Latitude
0,0,2020-01-04 14:39:00,2020,1,650 N 27TH ST,HANDGUN,4.0,14.0,3.0,14900.0,196.0,53208.0,2548652.700,384878.988,0,1,0,0,0,0,0,0,0,0,-87.947713,43.037556
1,2650072,2006-09-20 07:30:00,2006,9,1533 W WRIGHT ST,NaN,15.0,10.0,3.0,185500.0,148.0,53206.0,2552783.522,394664.536,0,0,0,0,0,0,0,0,1,0,-87.931358,43.064117
2,13230165,2010-11-19 20:45:00,2010,11,102 W BURLEIGH ST,GUN/HANDS,6.0,6.0,5.0,7000.0,122.0,53212.0,2558128.379,398704.395,0,1,0,0,0,0,0,0,0,0,-87.910985,43.074832
3,30823073,2007-05-02 09:05:00,2007,5,1010 S 2ND ST,NaN,12.0,16.0,2.0,186500.0,235.0,53204.0,2558178.583,379146.422,0,0,1,0,0,0,0,0,0,0,-87.912630,43.021183
4,40520170,2005-02-21 00:01:00,2005,2,2872 N 50TH ST,NaN,7.0,5.0,7.0,6000.0,161.0,53210.0,2540765.508,396454.197,0,0,0,0,0,0,0,0,0,1,-87.976155,43.069835


In [24]:
# Get unique years
unique_years = df['ReportedYear'].unique()

# Process each year
for year in unique_years:
    # Create a directory for the year
    year_dir = f"D:\QGIS Projects\MKE Crime Map\data\{year}"
    os.makedirs(year_dir, exist_ok=True)
    
    # Get the DataFrame for the specific year
    df_year = df[df['ReportedYear'] == year]
    
    # Loop through each crime type column (starting from 'Arson' to 'VehicleTheft')
    for crime in ['Arson', 'AssaultOffense', 'Burglary', 'CriminalDamage', 'Homicide', 'LockedVehicle', 'Robbery', 'SexOffense', 'Theft', 'VehicleTheft']:
        # Check if there are any incidents of this crime in the year
        if df_year[crime].sum() > 0:
            # Filter rows where this crime occurred
            crime_df = df_year[df_year[crime] == 1]
            
            # Convert to GeoDataFrame
            crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(crime_df.Longitude, crime_df.Latitude))
            crime_gdf.set_crs('epsg:4326', inplace=True)
            
            # Save GeoJSON file directly in the year folder
            crime_gdf.to_file(os.path.join(year_dir, f"{crime}_{year}.geojson"), driver='GeoJSON')